<a href="https://colab.research.google.com/github/amrit2603/Gen-AI/blob/main/RnnExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import tensorflow as tf
import numpy as np

In [45]:
def load_data(file_path):
  with open(file_path, 'r') as file:
    data = file.read()
    return data

file_path = '/content/01 Harry Potter and the Sorcerers Stone.txt'
text  = load_data(file_path).lower()

In [ ]:
text

In [47]:
#Tokenization

from tensorflow.keras.preprocessing.text import Tokenizer

Tokenizer = Tokenizer(oov_token='<OOV>')
Tokenizer.fit_on_texts(text)

total_words = len(Tokenizer.word_index) + 1

In [48]:
total_words

46

In [49]:
# Convert text to sequences
input_sequences = []
tokens = Tokenizer.texts_to_sequences([text])[0] # converts the input text into a list of numbers based on the word index

print(tokens)

[17, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1,

In [50]:
seq_length = 50  # Each input sequence contains 50 words
for i in range(seq_length, len(tokens)):
    input_sequences.append(tokens[i - seq_length:i + 1])


In [51]:
print(input_sequences[0])

[17, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [52]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [53]:
 input_sequences = np.array(pad_sequences(input_sequences, maxlen=seq_length + 1, padding='pre'))
 X = input_sequences[:, :-1]
 y = input_sequences[:, -1]

# after this X will have inputs and y will have label for those inputs


In [54]:
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [55]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN , Embedding , Dense

In [56]:
# Build the Simple RNN model
model = Sequential([
    Embedding(input_dim = total_words, output_dim=64),  # Word embeddings, removed input_length
    SimpleRNN(256, return_sequences=False),  # RNN Layer
    Dense(256, activation='relu'),  # Fully Connected Layer
    Dense(total_words, activation='softmax')  # Output Layer
])

In [57]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [58]:
model.fit(X , y , epochs=10 , batch_size = 128)

Epoch 1/10
633/633 ━━━━━━━━━━━━━━━━━━━━ 72s 110ms/step - accuracy: 0.9162 - loss: 0.4446
Epoch 2/10
633/633 ━━━━━━━━━━━━━━━━━━━━ 69s 109ms/step - accuracy: 0.9355 - loss: 0.3310
Epoch 3/10
633/633 ━━━━━━━━━━━━━━━━━━━━ 68s 107ms/step - accuracy: 0.9363 - loss: 0.3256
Epoch 4/10
633/633 ━━━━━━━━━━━━━━━━━━━━ 69s 109ms/step - accuracy: 0.9354 - loss: 0.3260
Epoch 5/10
633/633 ━━━━━━━━━━━━━━━━━━━━ 71s 111ms/step - accuracy: 0.9347 - loss: 0.3334
Epoch 6/10
633/633 ━━━━━━━━━━━━━━━━━━━━ 69s 109ms/step - accuracy: 0.9358 - loss: 0.3248
Epoch 7/10
633/633 ━━━━━━━━━━━━━━━━━━━━ 70s 111ms/step - accuracy: 0.9361 - loss: 0.3204
Epoch 8/10
633/633 ━━━━━━━━━━━━━━━━━━━━ 68s 108ms/step - accuracy: 0.9344 - loss: 0.3308
Epoch 9/10
633/633 ━━━━━━━━━━━━━━━━━━━━ 69s 109ms/step - accuracy: 0.9366 - loss: 0.3181
Epoch 10/10
633/633 ━━━━━━━━━━━━━━━━━━━━ 68s 107ms/step - accuracy: 0.9358 - loss: 0.3239


In [59]:
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")
print(f"First 5 elements of X[0]: {X[0][:5]}") # Displaying only first 5 tokens of first sequence for brevity
print(f"First 5 elements of X[1]: {X[1][:5]}")
print(f"First 5 elements of y[0]: {y[0][:5]}") # Displaying only first 5 elements of one-hot encoded vector
print(f"First 5 elements of y[1]: {y[1][:5]}")

Shape of X: (80972, 50)
Shape of y: (80972, 46)
First 5 elements of X[0]: [17  8  1  1  1]
First 5 elements of X[1]: [8 1 1 1 1]
First 5 elements of y[0]: [0. 1. 0. 0. 0.]
First 5 elements of y[1]: [0. 1. 0. 0. 0.]


In [60]:
# Function to generate text using RNN
def generate_text(seed_text, next_words=50):
    for _ in range(next_words):
        tokenized_input = Tokenizer.texts_to_sequences([seed_text])[0]
        tokenized_input = pad_sequences([tokenized_input], maxlen=seq_length, padding='pre')

        predicted_probs = model.predict(tokenized_input, verbose=0)
        predicted_index = np.argmax(predicted_probs)
        predicted_word = Tokenizer.index_word.get(predicted_index, "")

        seed_text += " " + predicted_word
    return seed_text


In [61]:
# Generate text using the trained model
print(generate_text("harry looked at"))

harry looked at <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV>


In [66]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense
import numpy as np

# Load and preprocess text
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    return text

file_path = '/content/01 Harry Potter and the Sorcerers Stone.txt'
text = load_data(file_path).lower()

# Tokenization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

tokenizer = Tokenizer(oov_token='') # Out-Of-Vocabulary token
                                        # If a word not seen during training appears later, it will be replaced with
                                        # Helps handle unknown words instead of ignoring them
tokenizer.fit_on_texts([text]) # analyzes the input text and creates a word index (mapping of words to unique integers)
total_words = len(tokenizer.word_index) + 1 #  0 is usually reserved for padding

# Convert text to sequences
input_sequences = []
tokens = tokenizer.texts_to_sequences([text])[0] # converts the input text into a list of numbers based on the word index
seq_length = 50  # Each input sequence contains 50 words

# First seq_length tokens (input): Used for training the model.
# Last token (target): Used as the label the model tries to predict.
# so total of (50 + 1) in one input_sequence index

for i in range(seq_length, len(tokens)):
    input_sequences.append(tokens[i - seq_length:i + 1])

# Pad sequences and split inputs/targets
# after this X will have inputs and y will have label for those inputs

input_sequences = np.array(pad_sequences(input_sequences, maxlen=seq_length + 1, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]

# One-hot encode the labels , note- there are other ways for
# encoding like pre-trained word2vec encoding and so on

y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Build the Simple RNN model
model = Sequential([
    Embedding(input_dim=total_words, output_dim=64, input_length=seq_length),  # Word embeddings
    SimpleRNN(256, return_sequences=False),  # RNN Layer
    Dense(256, activation='relu'),  # Fully Connected Layer
    Dense(total_words, activation='softmax')  # Output Layer
])

# 256 in RNN - The number of hidden units (size of the hidden state vector)
# return_sequences=False  - The RNN will only return the final hidden state after processing the entire sequence

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=30, batch_size=128)

# Function to generate text using RNN
def generate_text(seed_text, next_words=50):
    for _ in range(next_words):
        tokenized_input = tokenizer.texts_to_sequences([seed_text])[0]
        tokenized_input = pad_sequences([tokenized_input], maxlen=seq_length, padding='pre')

        predicted_probs = model.predict(tokenized_input, verbose=0)
        predicted_index = np.argmax(predicted_probs)
        predicted_word = tokenizer.index_word.get(predicted_index, "")

        seed_text += " " + predicted_word
    return seed_text

# Generate text using the trained model
print(generate_text("harry looked at"))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/30
633/633 ━━━━━━━━━━━━━━━━━━━━ 109s 167ms/step - accuracy: 0.0416 - loss: 6.9386
Epoch 2/30
633/633 ━━━━━━━━━━━━━━━━━━━━ 103s 163ms/step - accuracy: 0.0578 - loss: 6.3398
Epoch 3/30
633/633 ━━━━━━━━━━━━━━━━━━━━ 103s 163ms/step - accuracy: 0.0720 - loss: 6.1866
Epoch 4/30
633/633 ━━━━━━━━━━━━━━━━━━━━ 103s 163ms/step - accuracy: 0.0946 - loss: 5.8558
Epoch 5/30
633/633 ━━━━━━━━━━━━━━━━━━━━ 102s 162ms/step - accuracy: 0.1062 - loss: 5.6414
Epoch 6/30
633/633 ━━━━━━━━━━━━━━━━━━━━ 142s 162ms/step - accuracy: 0.1171 - loss: 5.4462
Epoch 7/30
633/633 ━━━━━━━━━━━━━━━━━━━━ 102s 162ms/step - accuracy: 0.1239 - loss: 5.2961
Epoch 8/30
633/633 ━━━━━━━━━━━━━━━━━━━━ 144s 166ms/step - accuracy: 0.1316 - loss: 5.1571
Epoch 9/30
633/633 ━━━━━━━━━━━━━━━━━━━━ 144s 169ms/step - accuracy: 0.1394 - loss: 5.0016
Epoch 10/30
633/633 ━━━━━━━━━━━━━━━━━━━━ 107s 170ms/step - accuracy: 0.1441 - loss: 4.8897
Epoch 11/30
633/633 ━━━━━━━━━━━━━━━━━━━━ 103s 163ms/step - accuracy: 0.1502 - loss: 4.7415
Epoch 12

The model learns local patterns, not long-term dependencies

RNNs struggle with long-range dependencies because they do not retain information well over long sequences.
This is why the text seems grammatically okay but lacks deeper context.
The model generates phrases based on probabilities

It predicts the most likely next word given the past words.
It does not understand meaning but follows statistical patterns.
It captures writing style but lacks coherence
Words appear logically related but do not form a strong narrative. The model does not truly "understand" the book, it just mimics word usage.**bold text**

In [67]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import numpy as np

# Function to load dataset
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    return text

# Load Harry Potter book text
file_path ='/content/01 Harry Potter and the Sorcerers Stone.txt'
text = load_data(file_path).lower()

# Tokenize the text
tokenizer = Tokenizer(oov_token='')
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Convert text into sequences
input_sequences = []
tokens = tokenizer.texts_to_sequences([text])[0]
seq_length = 50  # Each input sequence will have 50 words

for i in range(seq_length, len(tokens)):
    input_sequences.append(tokens[i - seq_length:i + 1])

# Pad sequences and split into inputs (X) and labels (y)
input_sequences = np.array(pad_sequences(input_sequences, maxlen=seq_length + 1, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)  # One-hot encode labels

# LSTM Model
model = Sequential([
    Embedding(input_dim=total_words, output_dim=100, input_length=seq_length),
    LSTM(256, return_sequences=True),  # First LSTM layer
    LSTM(256),  # Second LSTM layer
    Dense(total_words, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=20, batch_size=128)

# Function to Generate Text
def generate_text(seed_text, next_words=50, temperature=1.0):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=seq_length, padding='pre')

        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_probs = np.log(predicted_probs) / temperature  # Adjust randomness
        predicted_probs = np.exp(predicted_probs) / np.sum(np.exp(predicted_probs))
        predicted_index = np.random.choice(range(len(predicted_probs)), p=predicted_probs)

        output_word = tokenizer.index_word.get(predicted_index, "")
        seed_text += " " + output_word

    return seed_text

# Generate text
print(generate_text("harry looked at", next_words=50, temperature=0.7))


Epoch 1/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 619s 972ms/step - accuracy: 0.0427 - loss: 7.0754
Epoch 2/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 626s 989ms/step - accuracy: 0.0529 - loss: 6.3764
Epoch 3/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 628s 992ms/step - accuracy: 0.0657 - loss: 6.1233
Epoch 4/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 667s 969ms/step - accuracy: 0.0968 - loss: 5.8184
Epoch 5/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 639s 997ms/step - accuracy: 0.1130 - loss: 5.5532
Epoch 6/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 632s 999ms/step - accuracy: 0.1191 - loss: 5.3650
Epoch 7/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 636s 1s/step - accuracy: 0.1283 - loss: 5.1943
Epoch 8/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 622s 983ms/step - accuracy: 0.1340 - loss: 5.0537
Epoch 9/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 694s 1s/step - accuracy: 0.1429 - loss: 4.9422
Epoch 10/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 627s 991ms/step - accuracy: 0.1478 - loss: 4.8154
Epoch 11/20
633/633 ━━━━━━━━━━━━━━━━━━━━ 630s 994ms/step - accuracy: 0.1569 - loss: 4.6531
Epoch 12/20
63

In [68]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
import numpy as np

# Function to load dataset
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    return text

# Load Harry Potter book text
file_path = '/content/01 Harry Potter and the Sorcerers Stone.txt'
text = load_data(file_path).lower()

# Tokenize the text
tokenizer = Tokenizer(oov_token='')
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Convert text into sequences
input_sequences = []
tokens = tokenizer.texts_to_sequences([text])[0]
seq_length = 50  # Each input sequence will have 50 words

for i in range(seq_length, len(tokens)):
    input_sequences.append(tokens[i - seq_length:i + 1])

# Pad sequences and split into inputs (X) and labels (y)
input_sequences = np.array(pad_sequences(input_sequences, maxlen=seq_length + 1, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)  # One-hot encode labels

# GRU Model
model = Sequential([
    Embedding(input_dim=total_words, output_dim=100, input_length=seq_length),
    GRU(256, return_sequences=True),  # First GRU layer
    GRU(256),  # Second GRU layer
    Dense(total_words, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=20, batch_size=128)

# Function to Generate Text
def generate_text(seed_text, next_words=50, temperature=1.0):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=seq_length, padding='pre')

        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_probs = np.log(predicted_probs) / temperature  # Adjust randomness
        predicted_probs = np.exp(predicted_probs) / np.sum(np.exp(predicted_probs))
        predicted_index = np.random.choice(range(len(predicted_probs)), p=predicted_probs)

        output_word = tokenizer.index_word.get(predicted_index, "")
        seed_text += " " + output_word

    return seed_text

# Generate text
print(generate_text("harry looked at", next_words=50, temperature=0.7))

Epoch 1/20
 24/633 ━━━━━━━━━━━━━━━━━━━━ 8:33 843ms/step - accuracy: 0.0188 - loss: 8.4996

KeyboardInterrupt: 